In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import cv2
import numpy as np
import pywt
from sklearn.decomposition import PCA
from math import log10, sqrt
import os

# List files in the dataset folder to confirm the correct paths
dataset_path = '/kaggle/input/ct-and-mri/'
print(os.listdir(dataset_path))  # This will print all files in the dataset directory

# Load MRI and CT images (adjusted to correct paths)
mri_image = cv2.imread(dataset_path + 'mri_final.png', cv2.IMREAD_GRAYSCALE)
ct_image = cv2.imread(dataset_path + 'ct_final.png', cv2.IMREAD_GRAYSCALE)

# Check if the images are loaded correctly
if mri_image is None:
    print("Error loading MRI image!")
if ct_image is None:
    print("Error loading CT image!")

# Resize images if they are not the same size
mri_image = cv2.resize(mri_image, (256, 256))
ct_image = cv2.resize(ct_image, (256, 256))

# Step 1: Apply DWT to both images
def apply_dwt(image):
    coeffs = pywt.dwt2(image, 'haar')
    cA, (cH, cV, cD) = coeffs
    return cA, (cH, cV, cD)

mri_cA, (mri_cH, mri_cV, mri_cD) = apply_dwt(mri_image)
ct_cA, (ct_cH, ct_cV, ct_cD) = apply_dwt(ct_image)

# Step 2: Apply PCA on approximation coefficients to fuse them
def fuse_pca(cA1, cA2):
    pca = PCA(n_components=1)
    combined = np.stack((cA1.flatten(), cA2.flatten()), axis=1)
    pca_fused = pca.fit_transform(combined)
    return pca_fused.reshape(cA1.shape)

fused_cA = fuse_pca(mri_cA, ct_cA)

# Fuse detail coefficients by averaging
fused_cH = (mri_cH + ct_cH) / 2
fused_cV = (mri_cV + ct_cV) / 2
fused_cD = (mri_cD + ct_cD) / 2  # Step 3: Reconstruct the fused image using inverse DWT
fused_coeffs = fused_cA, (fused_cH, fused_cV, fused_cD)
fused_image = pywt.idwt2(fused_coeffs, 'haar')
fused_image = np.uint8(fused_image)  # Convert to uint8 for display

# Step 4: Calculate MSE
def calculate_mse(original, fused):
    mse = np.mean((original - fused) ** 2)
    return mse

mse_value = calculate_mse(mri_image, fused_image)

# Step 5: Calculate PSNR
def calculate_psnr(original, fused):
    mse = calculate_mse(original, fused)
    if mse == 0:
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

psnr_value = calculate_psnr(mri_image, fused_image)

# Display results
print(f"MSE between MRI and fused image: {mse_value}")
print(f"PSNR between MRI and fused image: {psnr_value} dB")

# Save fused image if needed
cv2.imwrite('/kaggle/working/fused_image.png', fused_image)

















/kaggle/input/ct-and-mri/ct_final.png
/kaggle/input/ct-and-mri/mri_final.png
['ct_final.png', 'mri_final.png']
MSE between MRI and fused image: 111.705322265625
PSNR between MRI and fused image: 27.650064950439166 dB


True